In [11]:
import json
import pandas as pd
import numpy as np

#TODO: Corregir calculo.

def load_data(profesores_file, salas_file):
    """Carga los datos de los archivos JSON"""
    with open(profesores_file, 'r', encoding='utf-8') as f:
        profesores = json.load(f)
    with open(salas_file, 'r', encoding='utf-8') as f:
        salas = json.load(f)
    return profesores, salas

def get_valid_timeslots_for_event(asignatura, total_timeslots=45):
    """Calcula los timeslots válidos para una asignatura considerando las restricciones"""
    nivel = asignatura['Nivel']
    horas = asignatura['Horas']
    
    # Inicializamos array de timeslots válidos
    valid_slots = np.ones(total_timeslots, dtype=bool)
    
    # División de días y bloques
    dias = 5
    bloques_por_dia = 9
    
    # Matriz de timeslots (días x bloques)
    timeslot_matrix = valid_slots.reshape(dias, bloques_por_dia)
    
    # Aplicar restricción de preferencia por nivel (años impares mañana, pares tarde)
    for dia in range(dias):
        if nivel % 2 == 1:  # Años impares
            timeslot_matrix[dia, 4:] = False  # Restricción para bloques de tarde
        else:  # Años pares
            timeslot_matrix[dia, :4] = False  # Restricción para bloques de mañana
            
    # Restricción de duración máxima (2 bloques consecutivos)
    for dia in range(dias):
        for bloque in range(bloques_por_dia - 2):
            # Si tres bloques consecutivos están disponibles, deshabilitamos el tercero
            if (timeslot_matrix[dia, bloque] and 
                timeslot_matrix[dia, bloque + 1] and 
                timeslot_matrix[dia, bloque + 2]):
                timeslot_matrix[dia, bloque + 2] = False
                
    # Convertimos la matriz de nuevo a un array
    valid_slots = timeslot_matrix.flatten()
    
    # Calculamos el número total de timeslots válidos
    valid_timeslots = np.sum(valid_slots)
    
    return valid_timeslots

def calculate_te_metrics(profesores, salas):
    """Calcula las métricas TE para todas las asignaturas"""
    total_timeslots = 45  # 5 días × 9 bloques
    results = []
    
    for profesor in profesores:
        for asignatura in profesor['Asignaturas']:
            # Calculamos timeslots válidos para la asignatura
            valid_timeslots = get_valid_timeslots_for_event(asignatura)
            
            # Calculamos TE
            te = valid_timeslots / total_timeslots
            
            # Guardamos resultados
            results.append({
                'Profesor': profesor['Nombre'],
                'Asignatura': asignatura['Nombre'],
                'CodigoAsignatura': asignatura['CodigoAsignatura'],
                'Nivel': asignatura['Nivel'],
                'Horas': asignatura['Horas'],
                'TimeslotsValidos': valid_timeslots,
                'TE': te
            })
    
    return pd.DataFrame(results)

def main():
    # Cargamos datos
    profesores, salas = load_data('../../agent_input/InputOfProfesores.json', '../../agent_input/InputOfSala.json')
    
    # Calculamos métricas TE
    df_results = calculate_te_metrics(profesores, salas)
    
    # Ordenamos por TE descendente
    df_results = df_results.sort_values('TE', ascending=False)
    
    # Calculamos estadísticas
    print("\nEstadísticas de TE:")
    print(f"Media: {df_results['TE'].mean():.3f}")
    print(f"Mediana: {df_results['TE'].median():.3f}")
    print(f"Desviación estándar: {df_results['TE'].std():.3f}")
    print(f"Mínimo: {df_results['TE'].min():.3f}")
    print(f"Máximo: {df_results['TE'].max():.3f}")
    
    # Guardamos resultados
    df_results.to_csv('TE_results.csv', index=False)
    print("\nResultados guardados en 'TE_results.csv'")
    
    # Mostramos los primeros resultados
    print("\nPrimeros 10 resultados:")
    print(df_results.head(10).to_string())

if __name__ == "__main__":
    main()


Estadísticas de TE:
Media: 0.409
Mediana: 0.444
Desviación estándar: 0.052
Mínimo: 0.333
Máximo: 0.444

Resultados guardados en 'TE_results.csv'

Primeros 10 resultados:
               Profesor                 Asignatura CodigoAsignatura  Nivel  Horas  TimeslotsValidos        TE
501  NEIRA CORTÉS PURIS  SISTEMAS INTEGRADOS DE GE        (ICA03-A)      0      3                20  0.444444
0    ROJAS RODRIGUEZ RO  SISTEMAS DE GESTIÓN DE CA        (PRI82-A)      8     12                20  0.444444
1    PADILLA GUZMAN SIL     DIRECCION DE PROYECTOS        (IC801-A)      8      2                20  0.444444
500  NEIRA CORTÉS PURIS  SISTEMAS INTEGRADOS DE GE        (ICA03-A)      0      1                20  0.444444
481  GANA BARRIENTOS FL                  ERGONOMÍA        (PRI64-S)      6     12                20  0.444444
479  CLARO COLLADO CORI             ÁLGEBRA LINEAL        (FIA22-B)      2      3                20  0.444444
478  CLARO COLLADO CORI             ÁLGEBRA LINEAL        (